In [4]:
import seaborn as sns
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
from decision_tree import DT
from classsification_gd import ClassificationGD
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score,auc,f1_score,precision_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.utils import resample

import matplotlib.pyplot as plt
import time



In [5]:
a = time.time()

[i for i in range(100000000)]

b = time.time()

print('Han pasado',(b - a),'segundos')

Han pasado 4.354077100753784 segundos


In [6]:
def update_class(X,main,other):
    result = []

    for i in range(len(X)):
        result.append(1 if X[i] == main else other)
        
    return np.array(result)

def to_multi_label(Y,clases):

    Y_result = []

    for class_ in Y:
        encode_for_elem = [0 for _ in range(len(clases.keys()))]
        encode_for_elem[clases[class_]] = 1
        Y_result.append(encode_for_elem)
    

    return np.array(Y_result)


class multi_svm:
    def __init__(self,alpha,epochs,c=10):
        self.models = []
        self.clases = []
        self.alpha = alpha
        self.epochs = epochs
        self.c = c

   
    def train(self,X_train,Y_train,X_test,Y_test,clases):
        self.models = []
        self.clases = clases 

        for _ in range(len(self.clases)):
            self.models.append(ClassificationGD(self.alpha,self.epochs,self.c,'svm'))
 

        for i in range(len(self.clases)):
            class_ = clases[i]
            Y_train_ = update_class(Y_train,class_,-1)
            Y_test_ = update_class(Y_test,class_,-1)

            self.models[i].train(X_train,Y_train_,X_test,Y_test_)
    
    def predict(self,X):


        predictions = []


        for point in X:

            scores = {self.clases[i]:self.models[i].svm_raw_predict(point) for i in range(len(self.models))}
            # print(scores)
            
            predictions.append(max(scores,key=scores.get))

        


        return np.array(predictions) 


In [7]:
class multi_logistic:
    def __init__(self,alpha,epochs,batch_size=30):
        self.models = []
        self.clases = []
        self.alpha = alpha
        self.epochs = epochs
        self.batch_size = batch_size

   
    def train(self,X_train,Y_train,X_test,Y_test,clases):
        self.models = []
        self.clases = clases 

        for _ in range(len(self.clases)):
            self.models.append(ClassificationGD(self.alpha,self.epochs))
 

        for i in range(len(self.clases)):
            class_ = clases[i]
            Y_train_ = update_class(Y_train,class_,0)
            Y_test_ = update_class(Y_test,class_,0)

            self.models[i].train(X_train,Y_train_,X_test,Y_test_,batch_size=self.batch_size)
    
    def predict(self,X):


        predictions = []


        for point in X:

            scores = {self.clases[i]:self.models[i].logistic_h(point) for i in range(len(self.models))}
            
            # print(scores)
            predictions.append(max(scores,key=scores.get))

        


        return np.array(predictions) 

In [14]:
def k_folds(X,Y,k,model):

    clases = list({elem for elem in Y})
    clases_map = {clases[i]:i for i in range(len(clases))}
        
    n_in_fold = len(Y)//k

    precision_ = []
    recall_ = []
    f1_score_ = []
    auc = []
    total_time = 0



    for i in range(k):
        print(i)

        X_train = np.concatenate([X[:i*n_in_fold],X[(i+1)*n_in_fold:]])
        Y_train = np.concatenate([Y[:i*n_in_fold],Y[(i+1)*n_in_fold:]])

        # print(X_train.shape)
        # print(Y_train.shape)


        X_test = X[i*n_in_fold:(i+1)*n_in_fold]
        Y_test = Y[i*n_in_fold:(i+1)*n_in_fold]


        # print(Y_test.shape)

        # print()
        # print()
        # print()
        initial_training_time = time.time()
        model.train(X_train,Y_train,X_test,Y_test,clases)
        training_time = time.time() - initial_training_time
        total_time += training_time 

        print(f'Para el fold {i} se demoro {training_time} para entrenar')

        pred = model.predict(X_test)
        one_hot_data_pred = to_multi_label(pred,clases_map)

        one_hot_real_data = to_multi_label(Y_test,clases_map)


        # print(confusion_matrix(Y_train,pred))
        precision_.append(precision_score(one_hot_real_data,one_hot_data_pred,average=None))
        recall_.append(recall_score(one_hot_real_data,one_hot_data_pred,average=None))
        f1_score_.append(f1_score(one_hot_real_data,one_hot_data_pred,average=None))
        # print(auc(Y_test,predicted_data))
    precision_ = np.sum(np.array(precision_),axis=0)/k
    recall_ = np.sum(np.array(recall_),axis=0)/k
    f1_score_ = np.sum(np.array(f1_score_),axis=0)/k


    print(f'Tiempo total: {total_time}')
    print(precision_,recall_,f1_score_)


        




In [72]:
def boostrap(X,Y,k,model):

    clases = list({elem for elem in Y})
    clases_map = {clases[i]:i for i in range(len(clases))}
        
    n_in_fold = len(Y)//k

    precision_ = []
    recall_ = []
    f1_score_ = []
    auc = []
    total_time = 0


    random_states = [42 + i for i in range(k)]



    for i in range(k):
        print(i)


        # while len(Y_test) == 0:

        mask_for_training = np.array([0 for i in range(len(Y))] )


        train_indexes = resample(range(len(Y)),n_samples=(len(Y)),replace=True,random_state=random_states[i])

        mask_for_training[train_indexes] = 1
        mask_for_testing = np.ones(len(Y)) - mask_for_training

        test_indexes = [i for i in range(len(Y)) if mask_for_testing[i]]
        # print(np.sum(mask_for_training))
        # print(np.sum(mask_for_testing))

        
        X_train = X[train_indexes]
        Y_train = Y[train_indexes]


        X_test = X[test_indexes]
        Y_test = Y[test_indexes]


        if len(Y_test) == 0:
            print(f'Para la seed {i} todo el testing es vacio. Continuando...')
            continue




        

        # print(Y_train.shape)
        # print(Y_test.shape)


        # print()
        # print()
        # print()
        initial_training_time = time.time()
        model.train(X_train,Y_train,X_test,Y_test,clases)
        training_time = time.time() - initial_training_time
        total_time += training_time 

        print(f'Para el boostrap {i} se demoro {training_time} para entrenar')

        pred = model.predict(X_test)
        one_hot_data_pred = to_multi_label(pred,clases_map)

        one_hot_real_data = to_multi_label(Y_test,clases_map)


        # print(confusion_matrix(Y_train,pred))
        precision_.append(precision_score(one_hot_real_data,one_hot_data_pred,average=None))
        recall_.append(recall_score(one_hot_real_data,one_hot_data_pred,average=None))
        f1_score_.append(f1_score(one_hot_real_data,one_hot_data_pred,average=None))
        # print(auc(Y_test,predicted_data))
    precision_ = np.sum(np.array(precision_),axis=0)/k
    recall_ = np.sum(np.array(recall_),axis=0)/k
    f1_score_ = np.sum(np.array(f1_score_),axis=0)/k


    print(f'Tiempo total: {total_time}')
    print(precision_,recall_,f1_score_)


        





0
Para el boostrap 0 se demoro 0.927595853805542 para entrenar
1
Para el boostrap 1 se demoro 0.8745002746582031 para entrenar
2
Para el boostrap 2 se demoro 0.8598225116729736 para entrenar
3
Para el boostrap 3 se demoro 0.9150900840759277 para entrenar
4


KeyboardInterrupt: 

In [64]:

scaler = MinMaxScaler()




data = pd.read_csv('./proyect_dataset/training.csv')
data[data.columns[:-1]] = scaler.fit_transform(data[data.columns[:-1]])
# data[data.columns[-1]] = update_class(data[data.columns[-1]],1,-1)
# data = data.to_numpy()

print(list({elem for elem in data[data.columns[-1]]}))
# print({elem for elem in update_class(data[data.columns[-1]],1,-1)})
# print(type(data['LB']))
# print(len(data['LB']))


data_train = data.sample(frac=0.7, random_state=43)
data_test = data[~data.index.isin(data_train.index)]




X = data.drop(columns='CLASE').to_numpy()
Y = data['CLASE'].to_numpy()


X_train = data_train.drop(columns='CLASE').to_numpy()
Y_train = data_train['CLASE'].to_numpy()

X_test = data_test.drop(columns='CLASE').to_numpy()
Y_test = data_test['CLASE'].to_numpy()




[1, 2, 3]


In [75]:

k_folds(X,Y,10,multi_logistic(0.15,1000,batch_size=30))


'''
Tiempo total: 8.472975492477417
[0.87322437 0.57422015 0.98333333] [0.98329474 0.3949355  0.33759396] [0.92471219 0.46236395 0.49981923]
'''

0
Para el fold 0 se demoro 0.8582112789154053 para entrenar
1
Para el fold 1 se demoro 0.8517951965332031 para entrenar
2
Para el fold 2 se demoro 0.8600203990936279 para entrenar
3
Para el fold 3 se demoro 0.8418850898742676 para entrenar
4
Para el fold 4 se demoro 0.868746280670166 para entrenar
5
Para el fold 5 se demoro 0.8430187702178955 para entrenar
6
Para el fold 6 se demoro 0.8290190696716309 para entrenar
7
Para el fold 7 se demoro 0.8361918926239014 para entrenar
8
Para el fold 8 se demoro 0.8373498916625977 para entrenar
9
Para el fold 9 se demoro 0.8467376232147217 para entrenar
Tiempo total: 8.472975492477417
[0.87322437 0.57422015 0.98333333] [0.98329474 0.3949355  0.33759396] [0.92471219 0.46236395 0.49981923]


In [80]:


k_folds(X,Y,10,multi_logistic(0.15,1000,batch_size=len(Y)))


'''Tiempo total: 211.1885039806366
[0.86592376 0.56470236 0.98333333] [0.98789498 0.33421913 0.35307015] [0.92255581 0.41236083 0.51718765]'''

0
Para el fold 0 se demoro 21.061896324157715 para entrenar
1
Para el fold 1 se demoro 20.325637340545654 para entrenar
2
Para el fold 2 se demoro 18.828165531158447 para entrenar
3
Para el fold 3 se demoro 18.51419496536255 para entrenar
4
Para el fold 4 se demoro 25.27824091911316 para entrenar
5
Para el fold 5 se demoro 21.7319118976593 para entrenar
6
Para el fold 6 se demoro 22.734477281570435 para entrenar
7
Para el fold 7 se demoro 21.02042531967163 para entrenar
8
Para el fold 8 se demoro 19.916369199752808 para entrenar
9
Para el fold 9 se demoro 21.777185201644897 para entrenar
Tiempo total: 211.1885039806366
[0.86592376 0.56470236 0.98333333] [0.98789498 0.33421913 0.35307015] [0.92255581 0.41236083 0.51718765]


In [79]:

boostrap(X,Y,30,multi_logistic(0.15,1000,batch_size=30))
# boostrap(X,Y,10,multi_logistic(0.15,1000,batch_size=30))


'''
Tiempo total: 26.17329692840576
[0.86535128 0.58420831 0.9765736 ] [0.98863957 0.34843651 0.34711507] [0.92278303 0.4339715  0.5097786 ]
'''

0
Para el boostrap 0 se demoro 0.9166812896728516 para entrenar
1
Para el boostrap 1 se demoro 0.8451783657073975 para entrenar
2
Para el boostrap 2 se demoro 0.8527379035949707 para entrenar
3
Para el boostrap 3 se demoro 0.8677055835723877 para entrenar
4
Para el boostrap 4 se demoro 0.8571021556854248 para entrenar
5
Para el boostrap 5 se demoro 0.8541038036346436 para entrenar
6
Para el boostrap 6 se demoro 0.8382079601287842 para entrenar
7
Para el boostrap 7 se demoro 0.8563787937164307 para entrenar
8
Para el boostrap 8 se demoro 0.9069242477416992 para entrenar
9
Para el boostrap 9 se demoro 0.9904742240905762 para entrenar
10
Para el boostrap 10 se demoro 0.9655158519744873 para entrenar
11
Para el boostrap 11 se demoro 0.9152321815490723 para entrenar
12
Para el boostrap 12 se demoro 0.9374470710754395 para entrenar
13
Para el boostrap 13 se demoro 0.8534829616546631 para entrenar
14
Para el boostrap 14 se demoro 0.8639559745788574 para entrenar
15
Para el boostrap 15 se demo

In [11]:
#initial_verification 
# alpha = 0.15
# epochs = 1000
# c = 10
# clases = list({elem for elem in data[data.columns[-1]]})

# # model = multi_svm(alpha,epochs,c)
# model = multi_logistic(alpha,epochs)
# model.train(X_train,Y_train,X_test,Y_test,clases)


# # model = DT(X_train,Y_train)





